In [1]:
import numpy as np
import math
import cv2
import os, os.path
import sys
import time
import pandas as pd
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph.opengl as gl

frames_dir = "kinect_data/"

#camera information based on the Kinect v2 hardware
CameraParams = {
  "cx":254.878,
  "cy":205.395,
  "fx":365.456,
  "fy":365.456,
  "k1":0.0905474,
  "k2":-0.26819,
  "k3":0.0950862,
  "p1":0.0,
  "p2":0.0,
}

# Kinect's physical orientation in the real world.
CameraPosition = {
    "x": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "y": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "z": 0, # height in meters of actual kinect sensor from the floor.
    "roll": 0, # angle in degrees of sensor's roll (used for INU input - trig function for this is commented out by default).
    "azimuth": 0, # sensor's yaw angle in degrees.
    "elevation": -30, # sensor's pitch angle in degrees.
}

def depthMatrixToPointCloudPos(z, scale=1000):
    #bacically this is a vectorized version of depthToPointCloudPos()
    C, R = np.indices(z.shape)

    R = np.subtract(R, CameraParams['cx'])
    R = np.multiply(R, z)
    R = np.divide(R, CameraParams['fx'] * scale)

    C = np.subtract(C, CameraParams['cy'])
    C = np.multiply(C, z)
    C = np.divide(C, CameraParams['fy'] * scale)

    return np.column_stack((z.ravel() / scale, R.ravel(), -C.ravel()))

def depthToPointCloudPos(x_d, y_d, z, scale=1000):
    # This runs in Python slowly as it is required to be called from within a loop, but it is a more intuitive example than it's vertorized alternative (Purly for example)
    # calculate the real-world xyz vertex coordinate from the raw depth data (one vertex at a time).
    x = (x_d - CameraParams['cx']) * z / CameraParams['fx']
    y = (y_d - CameraParams['cy']) * z / CameraParams['fy']

    return x / scale, y / scale, z / scale

def applyCameraOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # This runs slowly in Python as it is required to be called within a loop, but it is a more intuitive example than it's vertorized alternative (Purly for example)
    # use trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[ax1] ** 2 + pt[ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[ax2], pt[ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(0, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(1, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y plane

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

def applyCameraMatrixOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # bacically this is a vectorized version of applyCameraOrientation()
    # uses same trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[:, ax1] ** 2 + pt[:, ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[:, ax2], pt[:, ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[:, ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[:, ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(1, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(0, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
def process_frame(i):
    focal_x = 70.6
    depth_frame = np.load(frames_dir + str(i) + ".npy")
    obstacles = np.zeros(depth_frame.shape)
    img = depth_frame / 4500.
    imgray = np.uint8(img * 255)
    sure_bg_dilation = 0 #cv2.getTrackbarPos('bg_dilate', 'depth')
    sure_bg_erosion = 2 #cv2.getTrackbarPos('bg_erode', 'depth')
    thresh_dilation = 3 #cv2.getTrackbarPos('thresh_dilate', 'depth')
    thresh_erosion = 3 #cv2.getTrackbarPos('thresh_erode', 'depth')
    opening_iter = 3 #cv2.getTrackbarPos('opening', 'depth')
    unknown_erosion = 0 #cv2.getTrackbarPos('unknown_erode', 'depth')
    kernel_size = 5 #cv2.getTrackbarPos('kernel_size', 'depth')
    blur_amount = 0 #cv2.getTrackbarPos('blur', 'depth')
    if blur_amount % 2 == 0:
        blur_amount += 1
    ret,thresh = cv2.threshold(imgray,0,255,cv2.THRESH_BINARY)

    #noise removal
    kernel = np.ones((kernel_size,kernel_size),np.uint8)
    thresh = cv2.erode(thresh, kernel, iterations = thresh_erosion)
    thresh = cv2.dilate(thresh, kernel, iterations = thresh_dilation)
    opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = opening_iter)
    #gradient calculation
    gradient = cv2.morphologyEx(thresh, cv2.MORPH_GRADIENT, kernel)
    dilation = cv2.dilate(thresh,kernel,iterations = 1)
    #sure background area
    sure_bg = cv2.dilate(opening,kernel,iterations=sure_bg_dilation)
    sure_bg = cv2.erode(sure_bg, kernel, iterations = sure_bg_erosion)
    dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
    ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    #finding unknown region
    #unknown = cv2.subtract(gradient,sure_bg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    #unknown = cv2.subtract(unknown,gradient)
    unknown = cv2.medianBlur(unknown,blur_amount)
    unknown = cv2.erode(unknown, kernel, iterations = unknown_erosion)
    color = img#cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    #begin contour detection
    image, contours, hierarchy = cv2.findContours(sure_bg,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    color = cv2.drawContours(color, contours, -1, (0,255,0), 1)
    for cntr in contours:
        try:
            #calculate diamter of equivalent cirlce
            area = cv2.contourArea(cntr)
            equi_diameter = np.sqrt(4*area/np.pi)

            #Hardcoded Diameter Range in pixels
            LOW_DIAMETER_BOUND = 20
            HIGH_DIAMETER_BOUND = 150

            HIGH_DISTANCE_BOUND = 4500
            #Original tolerances were 20 and 150

            if(equi_diameter>LOW_DIAMETER_BOUND and equi_diameter<HIGH_DIAMETER_BOUND): #range needs to be tweaked
                mask = np.zeros_like(imgray)
                ellipse = cv2.fitEllipse(cntr)
                x,y,obj_length,obj_height = cv2.boundingRect(cntr)
                rect = cv2.minAreaRect(cntr)
                
                equi_diameter = obj_length
                
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                mask = cv2.ellipse(mask,ellipse,(255,255,255),-1)
                rows,cols = mask.shape
                #shift mask down to match obstacle, not edge
                M = np.float32([[1,0,0],[0,1,equi_diameter/4]])
                mask = cv2.warpAffine(mask,M,(cols,rows))
                mask = cv2.erode(mask, kernel, iterations=3)
                img_fg = cv2.bitwise_and(depth_frame,depth_frame,mask = mask)
                img_fg = cv2.medianBlur(img_fg,5)
                obstacles = cv2.add(np.float32(img_fg), np.float32(obstacles))



                # Experimenting with different blur settings
                #img_fg = cv2.GaussianBlur(img_fg, (5,5), 0)

                #mean_val = cv2.mean(img_fg)[0] #returns mean value of each channel, we only want first channel
                non_zero_mean = np.median(img_fg[img_fg.nonzero()])
                mean_val = non_zero_mean
                min_val, distance_to_object, min_loc, max_loc = cv2.minMaxLoc(img_fg)

                moment = cv2.moments(cntr)
                cx = int(moment['m10']/moment['m00'])
                cy = int(moment['m01']/moment['m00'])

                if mean_val < HIGH_DISTANCE_BOUND:
                    coords = depthToPointCloudPos(cx, cy, mean_val)
                    #coords = applyCameraOrientation(coords)
                    
                    mm_diameter = (equi_diameter) * (1.0 / CameraParams['fx']) * mean_val

                    img = cv2.ellipse(color,ellipse,(0,255,0),2)
                    cv2.drawContours(color,[box],0,(0,0,255),1)
                    cv2.rectangle(color,(x,y),(x+obj_length,y+obj_height),(0,255,0),2)
                    font = cv2.FONT_HERSHEY_SIMPLEX

                    cv2.putText(img, "x" + str(coords[0]), (cx,cy+30), font, 0.4, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.putText(img, "y" + str(coords[1]), (cx,cy+45), font, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.putText(color, "z" + str(mean_val), (cx,cy+60), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

                    cv2.putText(color,"diameter = " + str(mm_diameter), (cx,cy + 15), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

        except:
            print ("Failed to fit ellipse")

    
    cv2.imshow("unknown", unknown)
    cv2.imshow("obstacles", obstacles)
    cv2.imshow("depth", img)
    cv2.imwrite("original_depth_frame.png",depth_frame)
    cv2.imwrite("threshold.png",thresh)
    cv2.imwrite("opening.png",opening)
    cv2.imwrite("sure_bg.png",sure_bg)
    cv2.imwrite("distance_transform.png",dist_transform)
    cv2.imwrite("thresholded_dist_transform.png",sure_fg)
    cv2.imwrite("unknown.png",unknown)
    cv2.imwrite("processed_frame.png",img)
    key = cv2.waitKey(delay=1)
    #time.sleep(0.1)
    
    return obstacles

def plot_cloud(i, sp2, obstacles):
    depth_frame = np.load(frames_dir + str(i) + ".npy")
    img = depth_frame / 4500.
    xyz_arr = depthMatrixToPointCloudPos(depth_frame)
    xyz_arr = applyCameraMatrixOrientation(xyz_arr)
    obstacles_arr = depthMatrixToPointCloudPos(obstacles)
    obstacles_arr = applyCameraMatrixOrientation(obstacles_arr)
    colors = ((1.0, 1.0, 1.0, 1.0))
    colors = np.uint8(obstacles_arr)
    #colors = np.divide(colors, 255)
    #colors = colors.reshape(colors.shape[0] * colors.shape[1], 4 )
    colors = colors[:, :3:] #BGRA to BGR (slices out the alpha channel)  
    colors = colors[...,::-1] #BGR to RGB
    # Calculate a dynamic vertex size based on window dimensions and camera's position - To become the "size" input for the scatterplot's setData() function.
    v_rate = 5.0 # Rate that vertex sizes will increase as zoom level increases (adjust this to any desired value).
    v_scale = np.float32(v_rate) / gl_widget.opts['distance'] # Vertex size increases as the camera is "zoomed" towards center of view.
    v_offset = (gl_widget.geometry().width() / 1000)**2 # Vertex size is offset based on actual width of the viewport.
    v_size = v_scale + v_offset
    #cloud = PyntCloud(points)
    #cloud.plot(point_size=0.05, opacity=0.6)

    # Show the data in a scatter plot
    sp2.setData(pos=xyz_arr, color=colors, size=v_size)


In [5]:
def plane_fit(points):
    """
    p, n = planeFit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    from numpy.linalg import svd
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

def get_orientation(xyz_arr, n_iter):
    spherical = []
    planes = []
    centers = []
    for _ in range(0, n_iter):
        rand_points = []
        while len(rand_points) < 10:
            index = random.randrange(0,len(xyz_arr))
            if not xyz_arr[index].all() == np.zeros(3).all():
                rand_points.append(xyz_arr[index])
        rand_points = np.array(rand_points).T
        ctr, P = plane_fit(rand_points)
        r = math.sqrt(P[0]**2 + P[1]**2 + P[2]**2)
        theta = math.acos(P[2]/r) * 180 / math.pi
        phi = math.atan(P[1]/P[0]) * 180 / math.pi
        spherical.append(theta)
        planes.append(P)
        centers.append(ctr)
    return np.mean(centers, axis = 0), np.mean(planes, axis = 0), np.mean(spherical)
    

In [19]:
import numpy as np
import math
import random
import cv2
import os, os.path
import sys
import time
import pandas as pd
from pyqtgraph.Qt import QtCore, QtGui
import pyqtgraph.opengl as gl
from pyntcloud import PyntCloud
from pyntcloud.scalar_fields.base import ScalarField
from pyntcloud.scalar_fields.xyz import XYZScalarField
from pyntcloud.scalar_fields.xyz import PlaneFit
from IPython.display import display, HTML

def get_obstacles_with_plane(depth_frame, num_planes, max_distance):
    obstacles = np.zeros(depth_frame.shape) #empty image that will store the locations of detected obstacles
    img = np.uint8(depth_frame) #some opencv functions break when you pass a float array

    xyz_arr = depthMatrixToPointCloudPos(depth_frame) #convert depth data to XYZ coordinates
    print(xyz_arr.shape)
    center, plane, theta = get_orientation(xyz_arr, num_planes)
    CameraPosition['elevation'] = -theta
    center = applyCameraOrientation(center)
    plane = applyCameraOrientation(plane)
    xyz_arr = applyCameraMatrixOrientation(xyz_arr)
    #xyz_arr = np.array(filter(lambda x: x[1] > 0.3 + center[1], xyz_arr)) 
    #xyz_arr = xyz_arr[np.where(xyz_arr[2] > 0.3 - center[2])]
    #filtered = []
    #for point in xyz_arr:
    #    if point[2] > 0.1 - center[2]:
    #        filtered.append(point)
    plane_img = np.zeros(len(xyz_arr))
    plane_img[xyz_arr[:,2] > 0.12 - center[2]] = 1
    xyz_arr = xyz_arr[xyz_arr[:,2] > 0.12 - center[2]]
    points = pd.DataFrame(xyz_arr, columns=['x', 'y', 'z']) #convert XYZ coordinates to a DataFrame for pyntcloud
    cloud = PyntCloud(points)
    
    lines = [
        {
            "color": "red",
            "vertices": [[0, 0, 0], [10, 0, 0]]
        },
        {
            "color": "green",
            "vertices": [[0, 0, 0], [0, 10, 0]]
        },
        {
            "color": "blue",
            "vertices": [[0, 0, 0], [0, 0, 10]]
        },
        {
            "color": "pink",
            "vertices": [center.tolist(), plane.tolist()]
        }
    ]
    
    

    #start_time = time.time()
    #planes = [] #this will store the fitted planes
    #for i in range(0, num_planes):
    #    is_plane = cloud.add_scalar_field("plane_fit", max_dist=max_distance) #perform RANSAC plane fitting
    #    planes.append(np.float32(points['is_plane'])) #plane fitting adds an 'is_plane' column to our DataFrame 
    #duration = time.time() - start_time
    #print("Fitting took " + str(duration) + " seconds")

    #cloud.plot(use_as_color=is_plane, cmap="cool", polylines=lines)
    cloud.plot(cmap="cool", polylines=lines)

    #plane_img = np.mean(planes, axis = 0) #take the pixel average across all detected planes
    plane_img = np.uint8(np.reshape(plane_img,(424,512)) * 255) #reshape to match depth data and convert to uint8
    plane_img = np.uint8((np.ones((424,512)) * 255) - plane_img) #invert img so pixel value corresponds to NOT ground plane
    #plane_confidence = 0.75 #percentage of fitted planes that must have contained a certain point
    ret, plane_img = cv2.threshold(plane_img,0,255,cv2.THRESH_BINARY) #filter points that are probaly not ground plane
    plane_img = cv2.subtract(img, plane_img)
    #noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(plane_img ,cv2.MORPH_OPEN, kernel, iterations = 3) #erosion followed by dilation
    #sure_bg = cv2.erode(plane_img, kernel, iterations = 3) #further erosion to isolate obstacles

    color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) #BGR image to draw labels on

    #begin contour detection
    image, contours, hierarchy = cv2.findContours(opening,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    color = cv2.drawContours(color, contours, -1, (0,255,0), 1)
    for cntr in contours:
        try:
            #calculate diamter of equivalent circle
            #this measurement is only used for checking if countours fit our bounds
            area = cv2.contourArea(cntr)
            equi_diameter = np.sqrt(4*area/np.pi)

            #Hardcoded Diameter Range in pixels
            LOW_DIAMETER_BOUND = 20
            HIGH_DIAMETER_BOUND = 150

            HIGH_DISTANCE_BOUND = 4500
            #Original tolerances were 20 and 150

            if(equi_diameter>LOW_DIAMETER_BOUND and equi_diameter<HIGH_DIAMETER_BOUND):
                mask = np.zeros_like(img) #mask will contain the fitted and adjusted ellipse of a single obstacle
                ellipse = cv2.fitEllipse(cntr)
                x,y,obj_length,obj_height = cv2.boundingRect(cntr)
                rect = cv2.minAreaRect(cntr)

                equi_diameter = obj_length #bounding rectangle gives a better approximation of diameter

                box = cv2.boxPoints(rect)
                box = np.int0(box)
                mask = cv2.ellipse(mask,ellipse,(255,255,255),-1) #draw the fitted ellipse
                rows = mask.shape[0]
                cols = mask.shape[1]
                M = np.float32([[1,0,0],[0,1,equi_diameter/4]]) #shift mask down to match obstacle, not edge
                mask = cv2.warpAffine(mask,M,(cols,rows))
                mask = cv2.erode(mask, kernel, iterations=3) #erode the mask to remove background points
                img_fg = cv2.bitwise_and(depth_frame,depth_frame,mask = mask) #use the mask to isolate original depth values
                img_fg = cv2.medianBlur(img_fg,5) #median blur to further remove noise
                obstacles = cv2.add(np.float32(img_fg), np.float32(obstacles)) 
                
                mean_val = np.median(img_fg[img_fg.nonzero()]) #compute the non-zero average of obstacle depth values
                min_val, distance_to_object, min_loc, max_loc = cv2.minMaxLoc(img_fg)

                moment = cv2.moments(cntr) #get the centroid of the obstacle using its moment
                cx = int(moment['m10']/moment['m00'])
                cy = int(moment['m01']/moment['m00'])

                if mean_val < HIGH_DISTANCE_BOUND: #kinect loses accuracy beyond 4.5m
                    coords = depthToPointCloudPos(cx, cy, mean_val) #convert obstacle depth to XYZ coordinate

                    mm_diameter = (equi_diameter) * (1.0 / CameraParams['fx']) * mean_val #convert pixel diameter to mm

                    #begin visualization
                    cv2.drawContours(color,[box],0,(0,0,255),1)
                    cv2.rectangle(color,(x,y),(x+obj_length,y+obj_height),(0,255,0),2)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(color, "x" + str(coords[0]), (cx,cy+30), font, 0.4, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.putText(color, "y" + str(coords[1]), (cx,cy+45), font, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.putText(color, "z" + str(mean_val), (cx,cy+60), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(color,"diameter = " + str(mm_diameter), (cx,cy + 15), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

        except:
            print ("Failed to process image")
            print (sys.exc_info()[0])

    cv2.imshow("plane",plane_img)
    cv2.imshow("img",depth_frame)
    cv2.imshow("final",color)
    cv2.imshow("sure_bg",opening)
    cv2.imshow("obstacles",obstacles)
    key = cv2.waitKey(delay=1)
    return color



In [20]:
img = np.load('kinect_data/116.npy')
output = get_obstacles_with_plane(img, num_planes = 5, max_distance = 0.1)

(217088, 3)


C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\pythreejs\traits.py:175: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(0.009014453423769152, 0.49499956275137735, -…

In [15]:
frames_dir = 'kinect_data/'
directory = os.fsencode(frames_dir)
frame_i = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".npy"):
        file_path = frames_dir + filename
        print(file_path)
        img = np.load(file_path)
        output = get_obstacles_with_plane(img, num_planes = 5, max_distance = 0.1)
        cv2.imwrite(str(frame_i) + '.png', output)
        frame_i += 1
cv2.destroyAllWindows()

C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead
  after removing the cwd from sys.path.
C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\pyntcloud\geometry\models\plane.py:17: RuntimeWarning: invalid value encountered in true_divide
  self.normal = normal / np.linalg.norm(normal)
C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\pyntcloud\ransac\fitters.py:74: RuntimeWarning: invalid value encountered in less_equal
  inliers = all_distances <= model.max_dist


kinect_data/0.npy
Fitting took 5.690698146820068 seconds
kinect_data/1.npy


C:\Users\jumpr_000\AppData\Roaming\Python\Python35\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\jumpr_000\AppData\Roaming\Python\Python35\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in less


Fitting took 5.6419665813446045 seconds
kinect_data/10.npy
Fitting took 5.554112434387207 seconds
kinect_data/100.npy
Fitting took 5.603052139282227 seconds
kinect_data/101.npy
Fitting took 5.572788953781128 seconds
kinect_data/102.npy
Fitting took 5.562612533569336 seconds
kinect_data/103.npy
Fitting took 5.557041168212891 seconds
kinect_data/104.npy
Fitting took 5.572796821594238 seconds
kinect_data/105.npy
Fitting took 5.569042682647705 seconds
kinect_data/106.npy
Fitting took 5.573739051818848 seconds
kinect_data/107.npy
Fitting took 5.570455312728882 seconds
kinect_data/108.npy
Fitting took 5.547644853591919 seconds
kinect_data/109.npy
Fitting took 5.5537683963775635 seconds
kinect_data/11.npy
Fitting took 5.590807676315308 seconds
kinect_data/110.npy
Fitting took 5.55189061164856 seconds
kinect_data/111.npy
Fitting took 5.567464828491211 seconds
kinect_data/112.npy
Fitting took 5.553461074829102 seconds
kinect_data/113.npy
Fitting took 5.545739412307739 seconds
kinect_data/114.np

Fitting took 5.657552480697632 seconds
kinect_data/226.npy
Fitting took 5.852157115936279 seconds
kinect_data/227.npy
Fitting took 5.769652605056763 seconds
kinect_data/228.npy
Fitting took 5.82677698135376 seconds
kinect_data/229.npy
Fitting took 5.86987566947937 seconds
kinect_data/23.npy
Fitting took 5.737469434738159 seconds
kinect_data/24.npy
Fitting took 5.6930625438690186 seconds
kinect_data/25.npy
Fitting took 5.71571159362793 seconds
kinect_data/26.npy
Fitting took 5.653802871704102 seconds
kinect_data/27.npy
Fitting took 5.843183517456055 seconds
kinect_data/28.npy
Fitting took 5.921947002410889 seconds
kinect_data/29.npy
Fitting took 5.771690845489502 seconds
kinect_data/3.npy
Fitting took 5.772683143615723 seconds
kinect_data/30.npy
Fitting took 5.95632266998291 seconds
kinect_data/31.npy
Fitting took 5.884305000305176 seconds
kinect_data/32.npy
Fitting took 5.96824049949646 seconds
kinect_data/33.npy
Fitting took 5.906722545623779 seconds
kinect_data/34.npy
Fitting took 5.

def update():
    for i in range(0,229):
        print(i)
        obstacles = process_frame(i)
        plot_cloud(i,sp2,obstacles)
def update_image(i):
    plot_cloud(175,sp2,process_frame(175))
        
cv2.namedWindow('depth')

#cv2.createTrackbar('bg_dilate', 'depth', 0, 10, update_image) # 0
#cv2.createTrackbar('bg_erode', 'depth', 0, 10, update_image) # 2
#cv2.createTrackbar('thresh_dilate', 'depth', 0, 10, update_image) # 3
#cv2.createTrackbar('thresh_erode', 'depth', 0, 10, update_image) # 3
#cv2.createTrackbar('opening', 'depth', 0, 10, update_image) # 3
#cv2.createTrackbar('unknown_erode', 'depth', 0, 10, update_image) # 0
#cv2.createTrackbar('kernel_size', 'depth', 0, 10, update_image) # 5
#cv2.createTrackbar('blur', 'depth', 0, 10, update_image) # 0

#QT app
app = QtGui.QApplication([])
gl_widget = gl.GLViewWidget()
gl_widget.show()
gl_grid = gl.GLGridItem()
gl_widget.addItem(gl_grid)
#initialize some points data
pos = np.zeros((1,3))


sp2 = gl.GLScatterPlotItem(pos=pos)
sp2.setGLOptions('opaque') # Ensures not to allow vertexes located behinde other vertexes to be seen.

gl_widget.addItem(sp2)

t = QtCore.QTimer()
#t.timeout.connect(update)
t.start(50)

process_frame(200)
    
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()

sys.exit(0)


Raw Frame Viewer

import os
import cv2
import numpy as np
import time

frames_dir = "test_frames_5_6/"

directory = os.fsencode(frames_dir)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".npy"):
        cv2.imshow("test",np.load(frames_dir + '/'+ filename))
        key = cv2.waitKey(delay=30)
        print(filename)
        time.sleep(0.5)
        continue
    else:
        continue
cv2.destroyAllWindows()